In [1]:
import os
import numpy as np
import pandas as pd

import rdkit
import rdkit.Chem as Chem

In [5]:
sdf_file = 'alchemy/atom_9/1002198.sdf'

In [14]:
mol = Chem.SDMolSupplier(sdf_file)[0]

coords = np.full((9, 3), np.inf)

j = 0
for atom in mol.GetAtoms():
    if atom.GetSymbol() == 'H': continue
    pos = mol.GetConformer().GetAtomPosition(atom.GetIdx())  # Get coordinates for each atom
    coords[j] = pos.x, pos.y, pos.z
    j += 1

In [15]:
coords

array([[ 12.784 ,  23.8945, -35.3277],
       [ 12.6507,  24.1533, -33.8526],
       [ 12.2812,  23.2422, -32.9313],
       [ 11.8958,  21.8118, -33.1986],
       [ 12.2925,  23.5877, -31.4738],
       [ 12.1406,  22.7364, -30.6085],
       [ 12.5211,  24.9041, -31.1515],
       [ 12.3514,  25.9655, -32.1283],
       [ 13.033 ,  25.5557, -33.4285]])

In [14]:
with open(csv_fname, 'w') as f:
    f.write(','.join(labels) + '\n')

coordinates = []

for i, fname in tqdm(enumerate(os.listdir('xyz'))):
    fname = './xyz/' + fname

    with open(fname) as f: 
        lines = f.readlines()
    
    mol_id, *targets = lines[1].split()[1:]
    if mol_id in bad_samples: continue

    smiles = lines[-2].split()[-1:]

    # Generate Coordinates
    
    mol = Chem.MolFromSmiles(smiles[0])
    if mol is None: continue
    mol = Chem.AddHs(mol)
    embed = AllChem.EmbedMolecule(mol, randomSeed=16)  # Set a random seed for reproducibility
    if embed != 0: continue
    AllChem.UFFOptimizeMolecule(mol)  # Perform an optimization using UFF force field

    coords = np.full((9, 3), np.inf)

    j = 0
    for atom in mol.GetAtoms():
        if atom.GetSymbol() == 'H': continue
        pos = mol.GetConformer().GetAtomPosition(atom.GetIdx())  # Get coordinates for each atom
        coords[j] = pos.x, pos.y, pos.z
        j += 1

    coordinates.append(coords)
    
    with open(csv_fname, 'a') as f:
        f.write(','.join(smiles + targets) + '\n')

CC1=C(C)C(=O)NCC1
